In [1]:
# Function to calculate Incurred losses 

def calculate_IncurredLosses(dataframe,aggreagationtype,LossPaymentColName,CaseReservecolname,Aggreg_colname,
                             Claimnumbercolname,TransactionDate_colname,foryear,evaldate):
    ''' 
    Return incurred loss value 
    
    Parameters:
    dataframe : Dataframe is the pandas data frame which contains all the infor 
    aggreagationtype: can be 'policy','calendar','accident'
    LossPaymentColName : LossPayment column name in the data frame 
    CaseReservecolname : Case Reserve Value Column Name 
    Aggreg_colname : Aggregate year colname in the data frame 
    Claimnumbercolname : Claim Number to sort claims
    TransactionDate_colname : Date of Transaction
    foryear : aggregation for which particular year
    evaldate : Losses evaluated on which date 
    
    '''
    import pandas as pd
    verbose=False #Parameter to enable print statements
    ##Parameter check 
    #Dataframe 
    if(isinstance(dataframe, pd.DataFrame)):
        print("Pandas data frame is passed, good to go")
    else: 
        print("Please Pass a Pandas data frame ")
        
    #aggregation type 
    if(aggreagationtype in ['policy','calendar','accident','report']):
        print("The aggregation type is ",aggreagationtype)
    else:
        print("Please enter a valid aggregation type policy,calendar,report or accident")
    
    #Extract Transaction year from Transaction Date 
    dataframe['Transaction Year'] = pd.DatetimeIndex(dataframe[TransactionDate_colname]).year
    
    #Calculate the Case Incurred Loss = Loss Payment + Change in Case Reserve 
    dataframe.sort_values(by = [Claimnumbercolname,TransactionDate_colname])
    if(verbose):
        print("DataFrame After sorting values is above")
        print(display(dataframe))
    
    #Groupby And shift the Case Reserve Column to calculate Change in Case Reserve 
    CaseReserveShiftcolname = CaseReservecolname +'_shift1'
    
    dataframe[CaseReserveShiftcolname] =  (dataframe.groupby([Claimnumbercolname]))[CaseReservecolname].shift(1)
    dataframe.fillna(0,inplace=True)
    
    if(verbose):
        print("DataFrame After Case Reserve shift values is above")
        print(display(dataframe))
        
    #Calulate the Case Reserve Change Column 
    dataframe['Case Reserve Change'] =(dataframe[CaseReservecolname] - dataframe[CaseReserveShiftcolname])
    dataframe.drop(columns= [CaseReserveShiftcolname],inplace= True)
    if(verbose):
        print("DataFrame After Calculating Case Reserve Change values is above")
        print(display(dataframe))
        
    #Calculate Incurred Losses = Loss Payment + Change in Case Reserve 
    dataframe['Incurred Loss'] = dataframe[LossPaymentColName] + dataframe['Case Reserve Change']
    
    
    print("DataFrame with Incurred Loss Column is")
    print(display(dataframe))
         
    # Logic for aggregation : 
       
    if(aggreagationtype in ['accident','policy']):
        df_filter_onlyaccidents = dataframe[dataframe[Aggreg_colname] == dataframe['Transaction Year']]
        df_groupby_aggregate = df_filter_onlyaccidents.groupby(Aggreg_colname)['Incurred Loss'].sum()
        incurredloss_agg = df_groupby_aggregate.to_dict()[foryear]
    else: 
        
        df_groupby_aggregate = dataframe.groupby(Aggreg_colname)['Incurred Loss'].sum()
        incurredloss_agg = df_groupby_aggregate.to_dict()[foryear]
            
    return incurredloss_agg
        
    

In [2]:
# Function to calculate Exposures

def calculate_exposures(dataframe,exposureaggregation,exposuretype,Exposurecolname,EffectiveDate_colname,
                        ExpirationDate_colname, foryear,evaldate):
    ''' 
    Return exposure values

    Parameters:
    dataframe : Dataframe is the pandas data frame which contains all the infor 
    exposureaggregation: can be 'policy','calendar','accident'
    exposuretype : can be 'written','earned','inforce'
    Exposurecolname : Column name of base exposure
    EffectiveDate_colname : incurred loss column name in the data frame 
    ExpirationDate_colname : Aggregate year colname in the data frame 
    foryear : aggregation for which particular year
    evaldate : Losses evaluated on which date 
    
    '''
    import pandas as pd
    ##Parameter check 
    #Dataframe 
    if(isinstance(dataframe, pd.DataFrame)):
        print("Pandas data frame is passed, good to go")
    else: 
        print("Please Pass a Pandas data frame ")
        
    #aggregation type 
    if(exposureaggregation in ['policy','calendar','accident']):
        print("The aggregation type is ",exposureaggregation)
    else:
        print("Please enter a valid aggregation type policy,calendar or accident")
        
    #Exposure type 
    if(exposuretype in ['written','earned','inforce']):
        print("The exposure type is ",exposuretype)
    else:
        print("Please enter a valid exposure type written,earned or inforce")
        
    ## Check if effective date and Expiration date are date time columns if not convert them 
    #Finish this 
    alldatetimecolumns = [col for col in dataframe.columns if dataframe[col].dtype == 'datetime64[ns]']
    
    if((EffectiveDate_colname in alldatetimecolumns) & (ExpirationDate_colname in alldatetimecolumns)):
        pass
    elif(EffectiveDate_colname in alldatetimecolumns):
        print("Converting Expiration date into date time ")
        dataframe[ExpirationDate_colname] = pd.to_datetime(dataframe[ExpirationDate_colname], format = '%B %d,%Y')
    elif(ExpirationDate_colname in alldatetimecolumns):
        print("Converting Effective date into date time ")
        dataframe[EffectiveDate_colname] = pd.to_datetime(dataframe[EffectiveDate_colname], format = '%B %d,%Y')
    else:
        print("Converting Expiration date into date time ")
        print("Converting Effective date into date time ")
        dataframe[ExpirationDate_colname] = pd.to_datetime(dataframe[ExpirationDate_colname], format = '%B %d,%Y')
        dataframe[EffectiveDate_colname] = pd.to_datetime(dataframe[EffectiveDate_colname], format = '%B %d,%Y')
        
    #Extract effective year from effective date 
    dataframe['Effective Year'] = dataframe[EffectiveDate_colname].dt.year
    #Converting to Numerical date fromat 

    dataframe['Effective Time']=dataframe[ExpirationDate_colname]-dataframe[EffectiveDate_colname]
    dataframe['Effective Time in Years']= dataframe['Effective Time'].dt.days/360

    
    if(exposuretype=='written'):
        exp_eval_year = dataframe[dataframe['Effective Year'] == foryear][Exposurecolname].sum()
        #display(dataframe)
    elif(exposuretype=='earned'):
        if(aggreagationtype in ['policy','calendar']):
            dataframe['evaldate'] = evaldate
            dataframe['weightofexposure'] = (pd.to_datetime(dataframe['evaldate'])-dataframe[EffectiveDate_colname])/(dataframe['Effective Time'])
            maxVal=1
            dataframe['weightofexposure'][dataframe['weightofexposure'] >= maxVal] = maxVal #max value of weight is 1
            exp_eval_year = dataframe['weightofexposure'].dot(dataframe[Exposurecolname])
            #display(dataframe)
    elif(exposuretype == 'inforce'):
        #In force exposures is only for exposures which are effective on evaldate ,
        #some policies may not be inforce because its effective date is later
        dataframe['inforcedate'] = evaldate
        dataframe['inforcedate'] = pd.to_datetime(dataframe['inforcedate'])
        dataframe_inforce = dataframe[dataframe['inforcedate'] > dataframe[EffectiveDate_colname]]
        
        dataframe_inforce['weightofexposure'] = (pd.to_datetime(dataframe_inforce['inforcedate'])-dataframe_inforce[EffectiveDate_colname])/(dataframe_inforce['Effective Time'])
        maxVal=1
        dataframe_inforce['weightofexposure'][dataframe_inforce['weightofexposure'] >= maxVal] = maxVal #max value of weight is 1
        exp_eval_year = dataframe_inforce['weightofexposure'].dot(dataframe_inforce[Exposurecolname])
        display(dataframe_inforce)
   
    return exp_eval_year
    

In [3]:
#Utility functions for Variable Permissible Loss Ratio and Expense Ratios 

import pandas as pd
def RemoveincorrectExpenseRatio(x):
    return ((x if x<= 0.9 else ''))
def ConvertPercentagetoFloatCol(pdSeries):
    pdSeries=pdSeries.map(lambda x: x.lstrip('$').rstrip('%')) #Strip out % and any other unnecessary strings
    pdSeries = (pd.to_numeric(pdSeries,errors='coerce')).round(decimals=1)
    return pdSeries